# WEEK 6 - 

bisulfite seq!

In [ ]:
# Activate Enviornmant for week6 containing: anaconda fastqc bismark samtools bowtie2 igv

In [ ]:
# (cmdb) [~/qbb2020-answers/QUANT2020/data/lab6]conda activate week6

In [ ]:
Unzip
tar -xvzf methylation_fastq.tar.gz

In [ ]:
SRX1515395: GSM2027229: STEM-seq E4.0ICM rep1; Mus musculus; Bisulfite-Seq
SRR3083926_2.chr6.fastq
SRR3083926_1.chr6.fastq

SRX1515397: GSM2027231: STEM-seq E5.5Epi rep1; Mus musculus; Bisulfite-Seq
1 ILLUMINA (Illumina HiSeq 1500) run: 297.2M spots, 74.9G bases, 36Gb downloads
    SRR3083929_1.chr6.fastq
    SRR3083929_2.chr6.fastq

In [ ]:
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]bismark_genome_preparation --bowtie2 chr6
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]bismark --genome chr6/ -1 SRR3083926_1.chr6.fastq,SRR3083929_1.chr6.fastq -2 SRR3083926_2.chr6.fastq,SRR3083929_2.chr6.fastq


In [ ]:
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]samtools sort SRR3083929_1.chr6_bismark_bt2_pe.bam -o SRR29.sorted.bam
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]samtools sort SRR3083926_1.chr6_bismark_bt2_pe.bam -o SRR26.sorted.bam

In [ ]:
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]samtools index SRR26.sorted.bam
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]samtools index SRR29.sorted.bam

In [ ]:
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]bismark_methylation_extractor --bedgraph --comprehensive SRR26.sorted.bam.bai

In [ ]:
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]gunzip SRR3083926_1.chr6_bismark_bt2_pe.bedGraph.gz
(week6) [~/qbb2020-answers/QUANT2020/data/lab6]gunzip SRR3083929_1.chr6_bismark_bt2_pe.bedGraph.gz

# part 2! Lets do some scripting! 

In [125]:
E4 = open('/Users/cmdb/qbb2020-answers/QUANT2020/data/lab6/SRR3083926_1.chr6_bismark_bt2_pe.bedGraph', 'r') # Modify Phenotype file header and column #1
E5 = open('/Users/cmdb/qbb2020-answers/QUANT2020/data/lab6/SRR3083929_1.chr6_bismark_bt2_pe.bedGraph', 'r') # Modify Phenotype file header and column #1
Ref = open('/Users/cmdb/qbb2020-answers/QUANT2020/data/lab6/mm10_refseq_genes_chr6_50M_60M.bed')



In [83]:
EmbroD4=[] #parse day 4
for line in E4: 
    if "track" in line: # modify header
        continue
    else: #all the other lines
        line = line.split() # split by blanks
        EmbroD4.append(line)
E4.close()


In [85]:
EmbroD5=[] #parse day 5
for line in E5: 
    if "track" in line: # modify header
        continue
    else: #all the other lines
        line = line.split() # split by blanks 
        EmbroD5.append(line) #append back to a list
E5.close()

In [126]:
#create a dictionary of unique gene names using the 13th column as gene name and pointing to coordiants
unique_genes = {}
for line in Ref:
    gene_coord = list(map(int,line.split()[4:6])) #slpit on blanks by coodinants
    gene_name = line.split()[12] #split within the 13th column
    if gene_name not in unique_genes: #if key not found in dict
        unique_genes[gene_name] = gene_coord #add new key pointing to the coordinates
#print(unique_genes)
Ref.close()

In [123]:
def methylation_count(Embro, search_range): #use input of bedgraph data : Embro; list of genes: search_range
    meth_score = 0 # begin tally for the total meth
    count = 0 # count the number of sites
    for line in Embro:
        meth_range = list(map(int,line[1:3])) # nuc location
        score = float(line[3]) # Find score at cg site
        if meth_range[1]>=search_range[0] and meth_range[1]<=search_range[1]: # Is the location inside the gene?
            meth_score += score #add score of loci to total score
            count+=1 #iterate denominator
    if count != 0:
        return meth_score/count # Create an average
    else:
        return 0 # If there is no count add 0

In [129]:
# Maps genes to fold change from e4 to e5.5
foldC = open("foldchange.txt", "a")
D4_avg = open("day4_avg_meth.txt", "a")
D5_avg = open("day5_avg_meth.txt", "a")


for gene, coords in unique_genes.items(): #using the unique gene as a search reference - find methlation scores in each geen
    methscoresD4 = methylation_count(EmbroD4,coords) #write a score day 4 file 
    D4_avg.write(gene+ ":\t"+str(methscoresD4))
    D4_avg.write('\n')
    
    methscoresD5 = methylation_count(EmbroD5,coords)#write a score day 4 file 
    D5_avg.write(gene+ ":\t"+str(methscoresD5))
    D5_avg.write('\n')
    
    if methscoresD4 !=0: #only if methalation day 4 gene has a score - determine a fold change score
        foldC.write(gene+ ":\t"+str((methscoresD5-methscoresD4)/methscoresD4))
        foldC.write('\n')

        
#Close all the created files
foldC.close()
D4_avg.close()
D5_avg.close()